# Análisis de Datos - Programa Jóvenes a la E

Este notebook contiene análisis exploratorio y estadístico de los datos del programa Jóvenes a la E, enfocándose en diferentes dimensiones de equidad y cobertura.

## Objetivos del Análisis
1. Evaluar la cobertura y representatividad territorial
2. Analizar la distribución demográfica de beneficiarios
3. Examinar la participación por instituciones educativas
4. Identificar tendencias en la selección de programas académicos

## Estructura del Análisis

### 1. Análisis de Cobertura Territorial
- Distribución de postulaciones por localidad
- Identificación de zonas prioritarias
- Análisis de equidad territorial

### 2. Análisis Demográfico
- Distribución por grupos de edad
- Análisis por género
- Participación de grupos étnicos
- Análisis de población vulnerable

### 3. Análisis Institucional
- Distribución por instituciones educativas
- Modalidades de estudio preferidas
- Programas académicos más demandados

### 4. Análisis Temporal
- Tendencias por convocatoria
- Evolución de la participación
- Cambios en preferencias de programas

In [2]:
# Importar librerías
import mysql.connector as mysql
import pandas as pd
import logging as log

In [3]:
# Conectar a la base de datos
database_connection = mysql.connect(
    host='localhost',
    port=3306,
    user='root',
    password='adso00',
    database='jovenes_a_la_e'
)

if database_connection.is_connected():
    log.info("Conexión exitosa a la base de datos")


# 1. Análisis de Cobertura y Representatividad Territorial 
Verificar equidad territorial y focalización geográfica efectiva.

In [4]:
# Definir el cursor
cursor = database_connection.cursor()

# ¿Cuál es la distribución de beneficiarios por localidad?
cursor.execute("""
SELECT l.nombre_localidad, 
       COUNT(p.participacion_id) AS total_postulaciones,
       (COUNT(p.participacion_id) * 100.0 / (SELECT COUNT(*) FROM participacion)) AS porcentaje_postulaciones
FROM participacion p
JOIN estudiante e ON p.estudiante_id = e.estudiante_id
JOIN localidad l ON e.localidad_id = l.localidad_id
GROUP BY l.nombre_localidad
ORDER BY total_postulaciones DESC
""")

# Obtener nombres de columnas
columns = [desc[0] for desc in cursor.description]
# Crear DataFrame
df_q1 = pd.DataFrame(cursor.fetchall(), columns=columns)
# Mostrar como tabla
display(df_q1)

,nombre_localidad,total_postulaciones,porcentaje_postulaciones
0,BOSA,5124,13.86289
1,CIUDADBOLIVAR,4864,13.15946
2,KENNEDY,4513,12.20984
3,SUBA,4021,10.87874
4,USME,3474,9.39884
5,ENGATIVA,2927,7.91894
6,RAFAELURIBEURIBE,2578,6.97473
7,SANCRISTOBAL,2275,6.15497
8,USAQUEN,1125,3.04367
9,PUENTEARANDA,1118,3.02473


In [5]:
# ¿Cómo se relaciona el puntaje SISBEN con la localidad de residencia?
cursor.execute("""
SELECT l.nombre_localidad, 
       e.grupo_sisben, 
       COUNT(p.participacion_id) AS total_beneficiarios,
       (COUNT(p.participacion_id) * 100.0 / SUM(COUNT(p.participacion_id)) OVER (PARTITION BY l.nombre_localidad)) AS porcentaje_por_localidad
FROM participacion p
JOIN estudiante e ON p.estudiante_id = e.estudiante_id
JOIN localidad l ON e.localidad_id = l.localidad_id
GROUP BY l.nombre_localidad, e.grupo_sisben
ORDER BY l.nombre_localidad, porcentaje_por_localidad DESC;
""")

# Obtener nombres de columnas
columns = [desc[0] for desc in cursor.description]
# Crear DataFrame
df_q2 = pd.DataFrame(cursor.fetchall(), columns=columns)
# Mostrar como tabla
display(df_q2)

,nombre_localidad,grupo_sisben,total_beneficiarios,porcentaje_por_localidad
0,ANTONIONARIÑO,B - POBREZA MODERADA,139,33.01663
1,ANTONIONARIÑO,SIN SISBEN,127,30.16627
2,ANTONIONARIÑO,C - VULNERABLE,94,22.32779
3,ANTONIONARIÑO,A - POBREZA EXTREMA,47,11.16390
4,ANTONIONARIÑO,D - NO POBRE,14,3.32542
...,...,...,...,...
105,USME,B - POBREZA MODERADA,1506,43.35060
106,USME,C - VULNERABLE,907,26.10823
107,USME,A - POBREZA EXTREMA,627,18.04836
108,USME,SIN SISBEN,385,11.08233


In [6]:
# ¿Existen diferencias en el número de beneficiarios por género en cada localidad de Bogotá?
cursor.execute("""
SELECT l.nombre_localidad, 
       e.genero, 
       COUNT(p.participacion_id) AS total_beneficiarios,
       (COUNT(p.participacion_id) * 100.0 / SUM(COUNT(p.participacion_id)) OVER (PARTITION BY l.nombre_localidad)) AS porcentaje_por_localidad
FROM participacion p
JOIN estudiante e ON p.estudiante_id = e.estudiante_id
JOIN localidad l ON e.localidad_id = l.localidad_id
GROUP BY l.nombre_localidad, e.genero
ORDER BY l.nombre_localidad, porcentaje_por_localidad DESC
""")

columns = [desc[0] for desc in cursor.description]
df_q3 = pd.DataFrame(cursor.fetchall(), columns=columns)
display(df_q3)

,nombre_localidad,genero,total_beneficiarios,porcentaje_por_localidad
0,ANTONIONARIÑO,MUJER,272,64.60808
1,ANTONIONARIÑO,HOMBRE,149,35.39192
2,BARRIOSUNIDOS,MUJER,287,56.94444
3,BARRIOSUNIDOS,HOMBRE,217,43.05556
4,BOSA,MUJER,3099,60.48009
5,BOSA,HOMBRE,2025,39.51991
6,CHAPINERO,MUJER,198,61.49068
7,CHAPINERO,HOMBRE,124,38.50932
8,CIUDADBOLIVAR,MUJER,3035,62.39720
9,CIUDADBOLIVAR,HOMBRE,1828,37.58224


# 2. Acceso a la Educación Superior según condiciones sociales
Evaluar la inclusión y enfoque diferencial del programa.

In [7]:
# ¿Cómo es la  Distribución de beneficiarios por rango de edad según el género?

cursor.execute("""
SELECT e.genero, 
       e.rango_edad, 
       COUNT(p.participacion_id) AS total_beneficiarios,
       (COUNT(p.participacion_id) * 100.0 / SUM(COUNT(p.participacion_id)) OVER (PARTITION BY e.genero)) AS porcentaje_total
FROM estudiante e
JOIN participacion p ON e.estudiante_id = p.estudiante_id
GROUP BY e.genero, e.rango_edad
ORDER BY e.genero, total_beneficiarios DESC
""")

columns = [desc[0] for desc in cursor.description]
df_q4 = pd.DataFrame(cursor.fetchall(), columns=columns)
display(df_q4)

,genero,rango_edad,total_beneficiarios,porcentaje_total
0,HOMBRE,18-21 AÑOS,6900,47.45856
1,HOMBRE,MENOR DE 18 AÑOS,4247,29.21109
2,HOMBRE,22-25 AÑOS,2530,17.40147
3,HOMBRE,26-28 AÑOS,843,5.79820
4,HOMBRE,SIN INFORMACION,19,0.13068
5,INTERSEXUAL,18-21 AÑOS,4,66.66667
6,INTERSEXUAL,MENOR DE 18 AÑOS,2,33.33333
7,MUJER,18-21 AÑOS,9764,43.62629
8,MUJER,MENOR DE 18 AÑOS,6878,30.73142
9,MUJER,22-25 AÑOS,4018,17.95273


In [8]:
# ¿Cuál es la proporción de mujeres y hombres beneficiarios por año?

cursor.execute("""
SELECT e.genero, 
       COUNT(p.participacion_id) AS total_beneficiarios,
       (COUNT(p.participacion_id) * 100.0 / (SELECT COUNT(*) FROM participacion)) AS porcentaje
FROM estudiante e
JOIN participacion p ON e.estudiante_id = p.estudiante_id
GROUP BY e.genero              
""")

columns = [desc[0] for desc in cursor.description]
df_q5 = pd.DataFrame(cursor.fetchall(), columns=columns)
display(df_q5)

cursor.execute("""
SELECT c.anio, 
       e.genero, 
       COUNT(p.participacion_id) AS total_beneficiarios,
       (COUNT(p.participacion_id) * 100.0 / SUM(COUNT(p.participacion_id)) OVER (PARTITION BY c.anio)) AS porcentaje
FROM participacion p
JOIN estudiante e ON p.estudiante_id = e.estudiante_id
JOIN convocatoria c ON p.convocatoria_id = c.convocatoria_id
GROUP BY c.anio, e.genero
ORDER BY c.anio, porcentaje DESC;             
""")

columns2 = [desc[0] for desc in cursor.description]
df_q6 = pd.DataFrame(cursor.fetchall(), columns=columns2)
display(df_q6)

,genero,total_beneficiarios,porcentaje
0,HOMBRE,14539,39.33499
1,MUJER,22381,60.55138
2,SIN_DATO,36,0.09740
3,INTERSEXUAL,6,0.01623


,anio,genero,total_beneficiarios,porcentaje
0,2021,MUJER,6357,60.45074
1,2021,HOMBRE,4148,39.44466
2,2021,SIN_DATO,9,0.08558
3,2021,INTERSEXUAL,2,0.01902
4,2022,MUJER,6363,60.28994
5,2022,HOMBRE,4184,39.64374
6,2022,SIN_DATO,6,0.05685
7,2022,INTERSEXUAL,1,0.00948
8,2023,MUJER,6436,60.77432
9,2023,HOMBRE,4139,39.08404


In [9]:
# ¿Qué proporción de beneficiarios pertenece a grupos étnicos?

cursor.execute("""
SELECT e.grupo_etnico, 
       COUNT(p.participacion_id) AS total_beneficiarios,
       (COUNT(p.participacion_id) * 100.0 / (SELECT COUNT(*) FROM estudiante)) AS porcentaje_beneficiarios
FROM estudiante e
JOIN participacion p ON e.estudiante_id = p.estudiante_id
GROUP BY e.grupo_etnico
ORDER BY total_beneficiarios DESC
""")

columns = [desc[0] for desc in cursor.description]
df_q7 = pd.DataFrame(cursor.fetchall(), columns=columns)
display(df_q7)

,grupo_etnico,total_beneficiarios,porcentaje_beneficiarios
0,NINGUNO,33473,90.56058
1,COMUNIDADES NEGRAS O AFROCOLOMBIANAS RAIZALES ...,1457,3.94189
2,SIN INFORMACION,1201,3.24928
3,INDIGENA,829,2.24284
4,GITANO(A) O RROM,2,0.00541


In [10]:
# ¿Qué porcentaje de estudiantes son víctimas del conflicto armado o personas con discapacidad?

# Víctimas del conflicto armado
cursor.execute("""
SELECT 
    COUNT(*) AS total_victimas,
    (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM estudiante)) AS porcentaje_victimas
FROM estudiante
JOIN participacion p ON estudiante.estudiante_id = p.estudiante_id
WHERE victima_conflicto_arm = 'Sí'
""")

columns = [desc[0] for desc in cursor.description]
df_q8 = pd.DataFrame(cursor.fetchall(), columns=columns)
display(df_q8)

# Discriminación por género 
cursor.execute("""
SELECT e.genero, 
       COUNT(p.participacion_id) AS total_victimas,
       (COUNT(p.participacion_id) * 100.0 / (SELECT COUNT(*) FROM estudiante)) AS porcentaje_victimas
FROM estudiante e
JOIN participacion p ON e.estudiante_id = p.estudiante_id
WHERE e.victima_conflicto_arm = 'Sí'
GROUP BY e.genero
ORDER BY porcentaje_victimas DESC
""")

columns1 = [desc[0] for desc in cursor.description]
df_q9 = pd.DataFrame(cursor.fetchall(), columns=columns1)
display(df_q9)

# Discapacidad
cursor.execute("""
SELECT 
    COUNT(*) AS total_discapacitados,
    (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM estudiante)) AS porcentaje_discapacitados
FROM estudiante
JOIN participacion p ON estudiante.estudiante_id = p.estudiante_id
WHERE discapacidad = 'Sí'
""")

columns2 = [desc[0] for desc in cursor.description]
df_q10 = pd.DataFrame(cursor.fetchall(), columns=columns2)
display(df_q10)

,total_victimas,porcentaje_victimas
0,3823,10.34306


,genero,total_victimas,porcentaje_victimas
0,MUJER,2360,6.38494
1,HOMBRE,1462,3.95541
2,INTERSEXUAL,1,0.00271


,total_discapacitados,porcentaje_discapacitados
0,210,0.56815


In [11]:
# ¿Cómo se distribuyen por género los núcleos básicos del conocimiento academicos?

cursor.execute("""
SELECT pr.nombre_programa, 
       e.genero, 
       COUNT(p.participacion_id) AS total_inscritos,
       (COUNT(p.participacion_id) * 100.0 / SUM(COUNT(p.participacion_id)) OVER (PARTITION BY e.genero)) AS porcentaje_por_genero
FROM participacion p
JOIN estudiante e ON p.estudiante_id = e.estudiante_id
JOIN programa pr ON p.programa_id = pr.programa_id
GROUP BY pr.nombre_programa, e.genero
ORDER BY e.genero, porcentaje_por_genero DESC
""")

columns = [desc[0] for desc in cursor.description]
df_q11 = pd.DataFrame(cursor.fetchall(), columns=columns)
display(df_q11)

,nombre_programa,genero,total_inscritos,porcentaje_por_genero
0,ADMINISTRACIÓN,HOMBRE,1174,8.07483
1,SIN CLASIFICAR,HOMBRE,987,6.78864
2,"INGENIERÍA DE SISTEMAS, TELEMÁTICA Y AFINES",HOMBRE,873,6.00454
3,INGENIERÍA INDUSTRIAL Y AFINES,HOMBRE,746,5.13103
4,CONTADURÍA PÚBLICA,HOMBRE,693,4.76649
...,...,...,...,...
131,PUBLICIDAD Y AFINES,SIN_DATO,1,2.77778
132,QUÍMICA Y AFINES,SIN_DATO,1,2.77778
133,TERAPIAS,SIN_DATO,1,2.77778
134,ADMINISTRACIÓN,SIN_DATO,1,2.77778


# 3. Análisis de Calidad Académica de los Beneficiarios
Medir si el programa llega a estudiantes con mayores barreras educativas.

In [12]:
# ¿De que sector de colegio (Oficiales o Privados) hay más beneficiarios

cursor.execute("""
SELECT e.sector_colegio, 
       COUNT(p.participacion_id) AS total_beneficiarios,
       (COUNT(p.participacion_id) * 100.0 / (SELECT COUNT(*) FROM participacion)) AS porcentaje_beneficiarios
FROM estudiante e
JOIN participacion p ON e.estudiante_id = p.estudiante_id
GROUP BY e.sector_colegio
ORDER BY porcentaje_beneficiarios DESC
""")

columns = [desc[0] for desc in cursor.description]
df_q12 = pd.DataFrame(cursor.fetchall(), columns=columns)
display(df_q12)

,sector_colegio,total_beneficiarios,porcentaje_beneficiarios
0,OFICIAL,29140,78.83773
1,NO OFICIAL,4970,13.44624
2,NO APLICA,2852,7.71603


In [13]:
# ¿Cómo es la distrubución por zona de colegio (Urbana o rural) de los seleccionados?

cursor.execute("""
SELECT e.zona_colegio, 
       COUNT(p.participacion_id) AS total_beneficiarios,
       (COUNT(p.participacion_id) * 100.0 / (SELECT COUNT(*) FROM participacion)) AS porcentaje_beneficiarios
FROM estudiante e
JOIN participacion p ON e.estudiante_id = p.estudiante_id
GROUP BY e.zona_colegio
ORDER BY porcentaje_beneficiarios DESC
""")

columns = [desc[0] for desc in cursor.description]
df_q13 = pd.DataFrame(cursor.fetchall(), columns=columns)
display(df_q13)

,zona_colegio,total_beneficiarios,porcentaje_beneficiarios
0,URBANA,33242,89.93561
1,NO APLICA,2793,7.55641
2,RURAL,927,2.50798


In [14]:
# ¿Cómo es la distribución de los beneficiarios de acuerdo con el sisben?
cursor.execute("""
SELECT e.grupo_sisben, 
       COUNT(p.participacion_id) AS total_beneficiarios,
       (COUNT(p.participacion_id) * 100.0 / (SELECT COUNT(*) FROM estudiante)) AS porcentaje_beneficiarios
FROM estudiante e
JOIN participacion p ON e.estudiante_id = p.estudiante_id
GROUP BY e.grupo_sisben
ORDER BY porcentaje_beneficiarios DESC
""")

columns = [desc[0] for desc in cursor.description]
df_q14 = pd.DataFrame(cursor.fetchall(), columns=columns)
display(df_q14)

,grupo_sisben,total_beneficiarios,porcentaje_beneficiarios
0,B - POBREZA MODERADA,13716,37.10838
1,C - VULNERABLE,8875,24.01115
2,SIN SISBEN,7946,21.49775
3,A - POBREZA EXTREMA,5688,15.38878
4,D - NO POBRE,737,1.99394


# 4. Elección de Programa Académico y Modalidad
Entender preferencias formativas y posibles limitantes de acceso.

In [15]:
# ¿Cuáles son los núcleos básicos del conocimiento más seleccionados por los estudiantes?

cursor.execute("""
SELECT pr.nombre_programa, 
       COUNT(p.participacion_id) AS total_inscritos,
       (COUNT(p.participacion_id) * 100.0 / SUM(COUNT(p.participacion_id)) OVER ()) AS porcentaje_inscritos
FROM participacion p
JOIN programa pr ON p.programa_id = pr.programa_id
GROUP BY pr.nombre_programa
ORDER BY porcentaje_inscritos DESC
""")

columns = [desc[0] for desc in cursor.description]
df_q15 = pd.DataFrame(cursor.fetchall(), columns=columns)
display(df_q15)

,nombre_programa,total_inscritos,porcentaje_inscritos
0,ADMINISTRACIÓN,3144,8.50603
1,SIN CLASIFICAR,2584,6.99096
2,"INGENIERÍA DE SISTEMAS, TELEMÁTICA Y AFINES",2167,5.86278
3,INGENIERÍA INDUSTRIAL Y AFINES,1896,5.12959
4,ECONOMÍA,1729,4.67778
5,CONTADURÍA PÚBLICA,1696,4.58850
6,EDUCACIÓN,1484,4.01493
7,"INGENIERÍA ELECTRÓNICA, TELECOMUNICACIONES Y A...",1429,3.86613
8,DERECHO Y AFINES,1421,3.84449
9,"COMUNICACIÓN SOCIAL, PERIODISMO Y AFINES",1271,3.43867


In [16]:
# ¿Qué proporción de estudiantes elige modalidad virtual, presencial o a distancia?


cursor.execute("""
SELECT pr.modalidad, 
       COUNT(p.participacion_id) AS total_inscritos,
       (COUNT(p.participacion_id) * 100.0 / (SELECT COUNT(*) FROM participacion)) AS porcentaje_inscritos
FROM participacion p
JOIN programa pr ON p.programa_id = pr.programa_id
GROUP BY pr.modalidad
ORDER BY porcentaje_inscritos DESC
""")

columns = [desc[0] for desc in cursor.description]
df_q16 = pd.DataFrame(cursor.fetchall(), columns=columns)
display(df_q16)

,modalidad,total_inscritos,porcentaje_inscritos
0,PRESENCIAL,30576,82.72280
1,VIRTUAL,4325,11.70121
2,A DISTANCIA,1753,4.74271
3,PRESENCIAL-DUAL,172,0.46534
4,PRESENCIAL-VIRTUAL,136,0.36795


In [17]:
# ¿Qué programas academicos prefieren los beneficiarios de acuerdo a la zona?

# Zona rural
cursor.execute("""
SELECT pr.nombre_programa, 
       COUNT(p.participacion_id) AS total_inscritos,
       (COUNT(p.participacion_id) * 100.0 / SUM(COUNT(p.participacion_id)) OVER ()) AS porcentaje_inscritos
FROM participacion p
JOIN estudiante e ON p.estudiante_id = e.estudiante_id
JOIN programa pr ON p.programa_id = pr.programa_id
WHERE e.zona_colegio = 'Rural'
GROUP BY pr.nombre_programa
ORDER BY porcentaje_inscritos DESC
""")

columns = [desc[0] for desc in cursor.description]
df_q17 = pd.DataFrame(cursor.fetchall(), columns=columns)
display(df_q17)

# Zona urbana
cursor.execute("""
SELECT pr.nombre_programa, 
       COUNT(p.participacion_id) AS total_inscritos,
       (COUNT(p.participacion_id) * 100.0 / SUM(COUNT(p.participacion_id)) OVER ()) AS porcentaje_inscritos
FROM participacion p
JOIN estudiante e ON p.estudiante_id = e.estudiante_id
JOIN programa pr ON p.programa_id = pr.programa_id
WHERE e.zona_colegio = 'Urbana'
GROUP BY pr.nombre_programa
ORDER BY porcentaje_inscritos DESC
""")

columns = [desc[0] for desc in cursor.description]
df_q18 = pd.DataFrame(cursor.fetchall(), columns=columns)
display(df_q18)

,nombre_programa,total_inscritos,porcentaje_inscritos
0,ADMINISTRACIÓN,79,8.52211
1,SIN CLASIFICAR,64,6.90399
2,ECONOMÍA,49,5.28587
3,"INGENIERÍA DE SISTEMAS, TELEMÁTICA Y AFINES",47,5.07012
4,CONTADURÍA PÚBLICA,47,5.07012
5,INGENIERÍA INDUSTRIAL Y AFINES,46,4.96224
6,DERECHO Y AFINES,42,4.53074
7,PSICOLOGÍA,37,3.99137
8,"INGENIERÍA AMBIENTAL, SANITARIA Y AFINES",35,3.77562
9,"INGENIERÍA ELECTRÓNICA, TELECOMUNICACIONES Y A...",34,3.66775


,nombre_programa,total_inscritos,porcentaje_inscritos
0,ADMINISTRACIÓN,2825,8.49829
1,SIN CLASIFICAR,2304,6.93099
2,"INGENIERÍA DE SISTEMAS, TELEMÁTICA Y AFINES",1967,5.91721
3,INGENIERÍA INDUSTRIAL Y AFINES,1704,5.12605
4,ECONOMÍA,1553,4.67180
5,CONTADURÍA PÚBLICA,1513,4.55147
6,EDUCACIÓN,1346,4.04909
7,"INGENIERÍA ELECTRÓNICA, TELECOMUNICACIONES Y A...",1284,3.86258
8,DERECHO Y AFINES,1260,3.79039
9,"COMUNICACIÓN SOCIAL, PERIODISMO Y AFINES",1146,3.44745


# 5. Distribución por Institución de Educación Superior (IES)
Analizar el papel de las IES en el alcance del programa

In [18]:
# ¿Cuáles son las IES con mayor concentración de beneficiarios?

cursor.execute("""
SELECT i.nombre_inst_edu_superior, 
       COUNT(p.participacion_id) AS total_beneficiarios,
       (COUNT(p.participacion_id) * 100.0 / (SELECT COUNT(*) FROM participacion)) AS porcentaje_beneficiarios
FROM participacion p
JOIN programa pr ON p.programa_id = pr.programa_id
JOIN inst_edu_superior i ON pr.inst_edu_superior_id = i.inst_edu_superior_id
GROUP BY i.nombre_inst_edu_superior
ORDER BY total_beneficiarios DESC
""")

columns = [desc[0] for desc in cursor.description]
df_q19 = pd.DataFrame(cursor.fetchall(), columns=columns)
display(df_q19)

,nombre_inst_edu_superior,total_beneficiarios,porcentaje_beneficiarios
0,UNIVERSIDAD NACIONAL DE COLOMBIA,2191,5.92771
1,UNIVERSIDAD DE LOS ANDES,1718,4.64802
2,UNIVERSIDAD NACIONAL ABIERTA Y A DISTANCIA UNAD,1554,4.20432
3,UNIVERSIDAD EL BOSQUE,1550,4.19350
4,UNIVERSIDAD ANTONIO NARIÑO,1501,4.06093
5,UNIVERSIDAD SANTO TOMAS,1425,3.85531
6,UNIVERSIDAD DE LA SALLE,1261,3.41161
7,UNIVERSIDAD DISTRITAL-FRANCISCO JOSE DE CALDAS,1148,3.10589
8,UNIVERSIDAD SERGIO ARBOLEDA,1129,3.05449
9,UNIVERSIDAD EAN,1123,3.03826


In [19]:
# ¿Qué localidades están más asociadas a ciertas IES?

cursor.execute("""
SELECT l.nombre_localidad, 
       i.nombre_inst_edu_superior, 
       COUNT(p.participacion_id) AS total_beneficiarios,
       (COUNT(p.participacion_id) * 100.0 / SUM(COUNT(p.participacion_id)) OVER (PARTITION BY l.nombre_localidad)) AS porcentaje_localidad
FROM participacion p
JOIN estudiante e ON p.estudiante_id = e.estudiante_id
JOIN localidad l ON e.localidad_id = l.localidad_id
JOIN programa pr ON p.programa_id = pr.programa_id
JOIN inst_edu_superior i ON pr.inst_edu_superior_id = i.inst_edu_superior_id
GROUP BY l.nombre_localidad, i.nombre_inst_edu_superior
ORDER BY total_beneficiarios DESC
""")

columns = [desc[0] for desc in cursor.description]
df_q20 = pd.DataFrame(cursor.fetchall(), columns=columns)
display(df_q20)

,nombre_localidad,nombre_inst_edu_superior,total_beneficiarios,porcentaje_localidad
0,BOSA,UNIVERSIDAD NACIONAL DE COLOMBIA,321,6.26464
1,KENNEDY,UNIVERSIDAD NACIONAL DE COLOMBIA,279,6.18214
2,SUBA,UNIVERSIDAD NACIONAL DE COLOMBIA,252,6.26710
3,BOSA,UNIVERSIDAD NACIONAL ABIERTA Y A DISTANCIA UNAD,245,4.78142
4,CIUDADBOLIVAR,UNIVERSIDAD EL BOSQUE,244,5.01645
...,...,...,...,...
1066,TEUSAQUILLO,UNIVERSIDAD-COLEGIO MAYOR DE CUNDINAMARCA,1,0.37879
1067,TEUSAQUILLO,FUNDACION UNIVERSITARIA KONRAD LORENZ,1,0.37879
1068,TUNJUELITO,FUNDACION ESCUELA COLOMBIANA DE REHABILITACION,1,0.10277
1069,TUNJUELITO,CORPORACION UNIVERSIDAD PILOTO DE COLOMBIA,1,0.10277


In [20]:
# ¿Cómo afecta el grupo de Sisben al acceso a diferentes instituciones educativas?

cursor.execute("""
SELECT i.nombre_inst_edu_superior, 
       e.grupo_sisben, 
       COUNT(p.participacion_id) AS total_beneficiarios,
       (COUNT(p.participacion_id) * 100.0 / SUM(COUNT(p.participacion_id)) OVER (PARTITION BY i.inst_edu_superior_id)) AS porcentaje_sisben
FROM participacion p
JOIN estudiante e ON p.estudiante_id = e.estudiante_id
JOIN programa pr ON p.programa_id = pr.programa_id
JOIN inst_edu_superior i ON pr.inst_edu_superior_id = i.inst_edu_superior_id
GROUP BY i.nombre_inst_edu_superior, e.grupo_sisben, i.inst_edu_superior_id
ORDER BY i.nombre_inst_edu_superior, total_beneficiarios DESC
""")

columns = [desc[0] for desc in cursor.description]
df_q21 = pd.DataFrame(cursor.fetchall(), columns=columns)
display(df_q21)

# Cerrar el cursor y la conexión
cursor.close()
database_connection.close()

,nombre_inst_edu_superior,grupo_sisben,total_beneficiarios,porcentaje_sisben
0,COLEGIO MAYOR DE NUESTRA SEÑORA DEL ROSARIO,B - POBREZA MODERADA,384,38.63179
1,COLEGIO MAYOR DE NUESTRA SEÑORA DEL ROSARIO,C - VULNERABLE,233,23.44064
2,COLEGIO MAYOR DE NUESTRA SEÑORA DEL ROSARIO,SIN SISBEN,211,21.22736
3,COLEGIO MAYOR DE NUESTRA SEÑORA DEL ROSARIO,A - POBREZA EXTREMA,148,14.88934
4,COLEGIO MAYOR DE NUESTRA SEÑORA DEL ROSARIO,D - NO POBRE,18,1.81087
...,...,...,...,...
254,UNIVERSIDAD-COLEGIO MAYOR DE CUNDINAMARCA,B - POBREZA MODERADA,154,33.55120
255,UNIVERSIDAD-COLEGIO MAYOR DE CUNDINAMARCA,C - VULNERABLE,107,23.31155
256,UNIVERSIDAD-COLEGIO MAYOR DE CUNDINAMARCA,SIN SISBEN,106,23.09368
257,UNIVERSIDAD-COLEGIO MAYOR DE CUNDINAMARCA,A - POBREZA EXTREMA,76,16.55773
